# Notes Part 2

Overfitting - does *not* mean that you're training loss is lower than validation loss! 

A well fit will almost always have training loss lower than the validation loss.

Overfitting means - you're actually seeing your validation loss gettting worse.

### Five steps to avoid overfitting

More data

Data augmentation 

Generalizable architectures

Regularization

Reduce architecture complexity

### Steps to a basic modern CNN model

Matmul

Relu / Init

FC Forward

FC backward

Train loop

Conv

Optim

Batch-Norm

ResNet

If you want to develop your own library in Jupyter notebooks, follow notebook2script.py

In [ ]:
from fastai.basics import *
from pathlib import Path
from IPython.core.debugger import set_trace
from fastai import datasets
import pickle, gzip, math
import matplotlib as mpl
import torch
import matplotlib.pyplot as plt
from torch import tensor

# Get Data

In [ ]:
def get_data():
    path = "/work2/05515/bflynn/frontera/data/mnist.pkl.gz"
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

In [ ]:
def normalize(x, m, s): return (x-m)/s

In [ ]:
x_train,y_train,x_valid,y_valid = get_data()

In [ ]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

In [ ]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
# Why? If you normalize the two datasets with differently, they'll be on 
# totally different scales from one another - won't be able to tell 
# the differences between images in both sets 
x_valid = normalize(x_valid, train_mean, train_std)

In [ ]:
mpl.rcParams['image.cmap'] = 'gray'

In [ ]:
img = x_train[0]

In [ ]:
plt.imshow(img.view((28,28)));

# Matrix Multiplication

In [3]:
import time

def timeit(func):
    def wrapper(*args, **kwargs):
        now = time.time()
        retval = func(*args, **kwargs)
        print('{} took {:.5f}s'.format(func.__name__, time.time() - now))
        return retval
    return wrapper

In [3]:
# python only matrix multiplication

def matmul(a, b):
    ar, ac = a.shape
    br, bc = b.shape
    assert ac == br
    
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                c[i, j] += a[i,k] * b[k,j]
                
    return c

In [5]:
m1 = torch.rand((5, 784))
m2 = torch.rand((784, 10))

In [6]:
m1.shape, m2.shape

(torch.Size([5, 784]), torch.Size([784, 10]))

In [7]:
%timeit t = matmul(m1, m2)

922 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
t = matmul(m1, m2)

In [9]:
t.shape

torch.Size([5, 10])

About 1 second per matrix multiply, so if you have a 50,000 long dataset (MNIST), will take you 50,000 seconds 

### Every layer would take about 10 hours!

# How do we speed things up? Write in something other than python!

### PyTorch behind the scenes is using a library called a10

Elementwise operations

Operators +, -, *, >, <, == are usually element-wise

In [10]:
# Example

a = tensor([10., 6, -4])
b = tensor([2., 8, 7])
a,b 

(tensor([10.,  6., -4.]), tensor([2., 8., 7.]))

In [11]:
a + b

tensor([12., 14.,  3.])

In [12]:
(a < b).float().mean()

tensor(0.6667)

In [13]:
m = tensor([[1.,2,3], [4,5,6], [7,7,8]]); m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 7., 8.]])

## Frobenius norm:

$$\| A \|_F = \left( \sum_{i,j=1}^n | a_{ij} |^2 \right)^{1/2}$$

hint - you can copy LaTeX from wikipedia 

All it is:
**Matrix times itself, .sum(), .sqrt()**

In [14]:
(m*m).sum().sqrt()

tensor(15.9060)

In [19]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac == br
    
    c = torch.zeros(ar, bc)
    for i in range(ar):
        for j in range(bc):
            # TAKING OUT THE THIRD LOOP
            # REPLACING with colon, meaning entirety of that axis
            # i is the row number, j is the column
            # [i:, ] all of row i
            # [:,j] all of column j
            c[i, j] = (a[i,:] * b[:,j]).sum()
            
    return c

# this is running in C, not Python!

In [20]:
%timeit -n 10 _=matmul(m1, m2)

1.55 ms ± 76.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%timeit t =matmul(m1, m2)

1.35 ms ± 62.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Now have matrix multiplication around 63 microseconds

# Broadcasting

APL library

Use implicit broadcasted loops instead of for loops

In [22]:
a

tensor([10.,  6., -4.])

In [23]:
a > 0

tensor([ True,  True, False])

Element wise broadcasting
Broadcast a scalar to a tensor

The term **broadcasting** describes how arrays with different shapes are treated during arithmetic operations.  The term broadcasting was first used by Numpy.

From the [Numpy Documentation](https://docs.scipy.org/doc/numpy-1.10.0/user/basics.broadcasting.html):

    The term broadcasting describes how numpy treats arrays with 
    different shapes during arithmetic operations. Subject to certain 
    constraints, the smaller array is “broadcast” across the larger 
    array so that they have compatible shapes. Broadcasting provides a 
    means of vectorizing array operations so that looping occurs in C
    instead of Python. It does this without making needless copies of 
    data and usually leads to efficient algorithm implementations.
    
In addition to the efficiency of broadcasting, it allows us to write less code, which typically leads to fewer errors.

### Broadcasting acts like a for loop, and allows us to write at C speed

Broadcast one row accross multiple rows of a tensor super fast

In [24]:
a + 1

tensor([11.,  7., -3.])

In [25]:
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 7., 8.]])

In [26]:
m*2

tensor([[ 2.,  4.,  6.],
        [ 8., 10., 12.],
        [14., 14., 16.]])

In [28]:
c = tensor([10.,20,30]); c

tensor([10., 20., 30.])

In [29]:
t = c.expand_as(m)

In [30]:
t

tensor([[10., 20., 30.],
        [10., 20., 30.],
        [10., 20., 30.]])

In [31]:
t.storage()

 10.0
 20.0
 30.0
[torch.FloatStorage of size 3]

In [32]:
t.stride(), t.shape

((0, 1), torch.Size([3, 3]))

In [36]:
c # rank 1 tensor

tensor([10., 20., 30.])

In [37]:
c.unsqueeze(0) # rank 2 tensor

tensor([[10., 20., 30.]])

In [38]:
c.unsqueeze(1) # rank 2 tensor that looks like a column

tensor([[10.],
        [20.],
        [30.]])

In [39]:
c.shape, c.unsqueeze(0).shape, c.unsqueeze(1).shape

(torch.Size([3]), torch.Size([1, 3]), torch.Size([3, 1]))

In [41]:
c[None,:].shape, c[:None].shape

(torch.Size([1, 3]), torch.Size([3]))

In [43]:
c[None,:] # same as c.unsqueeze(0) - says please an axis in this row

tensor([[10., 20., 30.]])

In [45]:
c[:,None] # same as c.unsqueeze(1) - says please an axis in this column

tensor([[10.],
        [20.],
        [30.]])

In [46]:
c[:,None].expand_as(m) # Broadcast as columns

tensor([[10., 10., 10.],
        [20., 20., 20.],
        [30., 30., 30.]])

In [49]:
m

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 7., 8.]])

In [47]:
m + c[:, None]

tensor([[11., 12., 13.],
        [24., 25., 26.],
        [37., 37., 38.]])

In [48]:
m + c[None,:]

tensor([[11., 22., 33.],
        [14., 25., 36.],
        [17., 27., 38.]])

Matrix multiplication with Broadcasting

In [50]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar, bc)
    for i in range(ar):
#       c[i,j] = (a[i,:]          * b[:,j]).sum() # previous
        c[i]   = (a[i  ].unsqueeze(-1) * b).sum(dim=0) 
# unsqueeze(-1) or a[i, None]
# broadcasting - down 254 microseconds
    return c

In [51]:
# can replace every ;, in c[;, None] with a single . 
# this is convenient when you want to work with really high rank tensors

In [52]:
%timeit t = matmul(m1, m2)

297 µs ± 11.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Broadcasting Rules (same in numpy, pytorch and tensorflow)

When operating on two arrays/tensor, Numpy /Pytorch compares their shapes **element-wise**.

Starts with **trailing dimensions**, and works its way forward. Two dimensions are compatible when ...

- They are equal, or

- One of them is 1, in which case that dimension is broadcasted to make it the same size

Arrays do not need to have he same number of dimensions. 

For example, if you have a `256 * 256 * 3` RGB array, and you want to scale all colors of the image, you can multiply the entire image by a one-dim array with 3 values

Lining up the sizes of the trailing axes of these arrays according to the broadcast rules, shows that they are compatible:
    
``` Python
    Img: 256 x 256 x 3 (3d array)
    Scale: 3 (1d array)
    Result: 256 x 256 x 3 (3d array) # same as input!!!
```

# Einstein Summation

Einstein summation (`einsum`) is a compact representation for combining products and sums in a general way. From the numpy docs:

"The subscripts string is a comma-separated list of subscript labels, where each label refers to a dimension of the corresponding operand. Whenever a label is repeated it is summed, so `np.einsum('i,i', a, b)` is equivalent to `np.inner(a,b)`. If a label appears only once, it is not summed, so `np.einsum('i', a)` produces a view of a with no changes."

In [61]:
def matmul(a,b): return torch.einsum('ik,kj -> ij', a, b)

In [57]:
# c[i,j] += a[i,k] * b[k,j]

#   ik, kj -> ij

# c[i,j] = (a[i,:] * b[:,j]).sum()

input has i rows, output has i rows
input has j columns, output has j columns

k is what we're mutliplying the rows and the columns by

In [58]:
# any place where something is repeated, and take the dot product over them
# 'ik, kj -> ij'

In [59]:
%timeit t = matmul(m1, m2)

75.6 µs ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
# say we needed batchwise matrix multiplication??

def matmul(a,b): return torch.einsum('bik,bkj -> bij', a, b)

# Pytorch Operation

use pytorch's function or operator directly for matrix multiplication!

In [62]:
%timeit t = m1.matmul(m2) 

12.2 µs ± 5.5 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
from fastai.basics import *

From regular Python to C in Pytorch, we went from 922 ms to 12.2 µs!!!

In [5]:
# if not exist, download mnist dataset
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim

In [6]:
n = 50000
m = 784
c = tensor(10)
nh = 50 # number of hidden

In [7]:
# simplified kaiming init / he init (NEED TO DIVIDE BY SQRT M)
w1 = torch.randn(m,nh)/math.sqrt(m) # weight matrices
b1 = torch.zeros(nh) 
w2 = torch.randn(nh,1)/math.sqrt(m)
b2 = torch.zeros(1)

In [9]:
def lin(x, w, b): return x@w + b

Initializations are really important! Can make a huge difference in overall performance

Just removed everything that was negative, but that does not mean that you make the standard deviation one and mean zero

## Delving deep into rectifiers

Two initialization strategies you'll see all the time: Kaiming and Glorot 

This is the Kaiming Paper

Reading papers from competition winners - Chocked full of lots of good ideas to try!

Section 2.2 of the paper

Initialization of Filter Weights for Rectifiers

Rectifier: rectified linear unit. It is any neural network with rectified linear units in it

People used to initialize models with Gaussian (normal) distributions

Tend to not train very well, because reduces the variance each layer to a point where you lose everything as the network gets deeper

In [ ]:
t = lin(x_valid, w1, b1)

t.mean(),t.std()

In [ ]:
def relu(x): return x.clamp_min(0.)

t = relu(lin(x_valid, w1, b1)) ## how the first layer is defined
# relu = replace all negatives with zeros!

#...actually it really should be this!
t.mean(),t.std()

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

## Read 2.2 of Kaiming et al 

In [ ]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') # fan out is to account for the way that 
# pytorch transposes the matrix prior to doing a matrix multiplication 
# (check the source code)

In [17]:
# can make a model with linear, relu, and linear again

def model(xb):
    l1 = lin(xb, w1, b1) # linear layer
    l2 = relu(l1) # relu - remember, just everything that's negative gets turned into zero
    l3 = lin(l2, w2, b2) # another linear layer
    # all are connected and go forward - result of lin gets passed as input to relu, and the result of relu gets passed to the third linear layer
    return l3

In [18]:
#export

# squeeze - opposite of unsqueeze - gets rid of unit axis
# squeeze - call THE DIMENSION (1 or -1), so doesn't break, more resilient
# more resilient to batch size 1
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

# Forward pass completed
### However, a forward pass is useless without a backward pass

The backward pass is what tells us how to update our parameters; and we need gradients in order to tell us how *much* to update our parameters 

### Need to know the chain rule!

```Python
y_hat = mse(lin2(relu(lin1(x))), y) 
# where y is the target
```

Need to find the gradient wiht respect to the parameters

y = f(u) 

u = g(x)

dy/dx = dy/du * du/dx 

### Derivative of each function on its own, and then multiply them all together! 

Any time you see a derivative, can cross multiply! This is the calculus of infintesimals!

Just think of it as a small change in y over a small change in x

## Chain rule from scratch in code

Start from the outer most layer of the function, the loss function mse

In [ ]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [ ]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    # gradient of a matrix product is the matrix product 
    # with the transpose! 
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

## Going from most outward to most inward layer of the function

In [ ]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ) # pass in results of forward pass
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2) # also has access to the grad of next layer
    lin_grad(inp, l1, w1, b1)
    
    # the loss never appears in the gradients! Never used as input

## Back propagation is just the chain rule, where we save the intermediate calculations, so we don't have to calculate them again

# Refactor 

### Layers as classes

dunder call - now can treat the class as a function

save the input, save output, and return the output

Backward: backward pass, but now saved inside self.input.gradient

In [20]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [21]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [22]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [23]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

## Module class structure

In [24]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [25]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [26]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        # using einsum!!! much faster now
        self.b.g = out.g.sum(0)

In [27]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [28]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

## nn.Linear and nn.Module

In [29]:
from torch import nn 

In [30]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [31]:
model = Model(m, nh, 1)